In [29]:
import numpy as np

class Leastsq:
    def __init__(self, func, jac, x0, y, xdata, bounds=None, maxiter=500, tol=1e-6, mu=1e-3, nu=2):
        self.func = func
        self.jac = jac
        self.x = np.array(x0)
        self.y = np.array(y)
        self.xdata = np.array(xdata)
        self.bounds = bounds
        self.maxiter = maxiter
        self.tol = tol
        self.mu = mu
        self.nu = nu
        self.iter = 0

    def objective_function(self, params):
        y_pred = self.func(self.xdata, *params)
        error = self.y - y_pred
        return np.sum(error**2)
    
    def solve(self):
        while self.iter < self.maxiter:
            # Compute the Jacobian
            J = self.jac(*self.x)

            # Compute the residual
            r = self.y - self.func(self.xdata, *self.x)

            # Compute the cost function
            cost = np.dot(r, r)

            # Compute the gradient
            grad = np.dot(J.T, r)

            # Compute the Hessian
            H = np.dot(J.T, J)

            # Add damping parameter to Hessian
            H_lm = H + self.mu * np.identity(len(self.x))

            # Compute the step
            step = np.linalg.solve(H_lm, -grad)

            # Update x
            x_new = self.x + step

#             if self.bounds is not None:
#                 # Enforce bounds on parameters
#                 lb, ub = self.bounds
#                 x_new = np.maximum(np.minimum(x_new, ub), lb)

            # Compute the new residual
            r_new = self.y - self.func(self.xdata, *x_new)

            # Compute the new cost function
            cost_new = np.dot(r_new, r_new)

            # Compute the ratio of actual reduction to predicted reduction
            rho = (cost - cost_new) / (np.dot(step, grad) + 0.5 * np.dot(step, np.dot(H_lm, step)))

            if rho > 0:
                # Actual reduction is positive, so accept the step
                self.x = x_new
                self.mu = self.mu * max(1 / 3, 1 - (2 * rho - 1) ** self.nu)
                self.iter += 1
                if np.linalg.norm(step) < self.tol:
                    break
            else:
                # Actual reduction is negative, so reject the step
                self.mu = self.mu * 2



In [30]:
# Load data
xdata = np.loadtxt('xdata_mask20.txt')
ydata = np.loadtxt('ydata_mask20.txt')

# Define the function to fit
def func(x, a, b, omega, phi, c):
    return a * np.exp(-b * x) * np.sin(omega * x + phi) + c

# Define the Jacobian
# def jac(a, b, omega, phi, c):
#     J = np.zeros((len(xdata), 5))
#     J[:, 0] = np.exp(-b * xdata) * np.sin(omega * xdata + phi)
#     J[:, 1] = -a * xdata * np.exp(-b * xdata) * np.sin(omega * xdata + phi)
#     J[:, 2] = a * xdata * np.exp(-b * xdata) * np.cos(omega * xdata + phi)
#     J[:, 3] = a * np.exp(-b * xdata) * np.cos(omega * xdata + phi)
#     J[:, 4] = np.ones(len(xdata))
#     return J

def jac(x, a, b, omega, phi, c):
    dfda = np.exp(-b * x) * np.sin(omega * x + phi)
    dfdb = -a * x * np.exp(-b * x) * np.sin(omega * x + phi)
    dfdom = a * x * np.exp(-b * x) * np.cos(omega * x + phi)
    dfdphi = a * np.exp(-b * x) * np.cos(omega * x + phi)
    dfdc = np.ones_like(x)
    return np.column_stack((dfda, dfdb, dfdom, dfdphi, dfdc))


# Define the initial estimate of the parameters
x0 = [1, 1, 1, 1, 1]
# Define the bounds for the parameters
bounds = (0,5)
# Instantiate the Leastsq class
lsq = Leastsq(func, jac, x0, ydata, xdata, bounds=bounds)

# Run the solver
lsq.solve()

# Access the final estimate of the parameters
params = lsq.x
print(params)


TypeError: jac() missing 1 required positional argument: 'c'